In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read data

item_categories = pd.read_csv('Data/item_categories.csv')
items = pd.read_csv('Data/items.csv')
shops = pd.read_csv('Data/shops.csv')
sales = pd.read_csv('Data/sales_train_v2.csv')
submission = pd.read_csv('submission.csv')
test = pd.read_csv('Data/test.csv')

In [3]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
print(test.shape)
print(sales.shape)

(214200, 3)
(2935849, 6)


In [6]:
submission.head()

,ID,item_cnt_month
0,0,1.263679
1,1,3.577624
2,2,1.376003
3,3,0.929417
4,4,0.526681


In [7]:
submission['floor_item_cnt_month'] = submission.item_cnt_month.apply(np.floor)
submission['date_block_num'] = 34
submission.head()

,ID,item_cnt_month,floor_item_cnt_month,date_block_num
0,0,1.263679,1.0,34
1,1,3.577624,3.0,34
2,2,1.376003,1.0,34
3,3,0.929417,0.0,34
4,4,0.526681,0.0,34


In [8]:
out = test.merge(submission)
out.head()

,ID,shop_id,item_id,item_cnt_month,floor_item_cnt_month,date_block_num
0,0,5,5037,1.263679,1.0,34
1,1,5,5320,3.577624,3.0,34
2,2,5,5233,1.376003,1.0,34
3,3,5,5232,0.929417,0.0,34
4,4,5,5268,0.526681,0.0,34


In [9]:
month = sales.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum()
month.head()

shop_id  item_id  date_block_num
0        30       1                 31.0
         31       1                 11.0
         32       0                  6.0
                  1                 10.0
         33       0                  3.0
Name: item_cnt_day, dtype: float64

In [10]:
print(month.shape)
print(out.shape)
print(month[5][5232])

(1609124,)
(214200, 6)
date_block_num
31    1.0
Name: item_cnt_day, dtype: float64


In [11]:
print(month[0][32].index)
print(out.shape)

Int64Index([0, 1], dtype='int64', name=u'date_block_num')
(214200, 6)


In [12]:
length = out.shape[0]
k = length
ID = []
shop_id = []
item_id = []
count = []
dbn = []
for i in range(length):
    val = out.iloc[i]
    if i%1000 == 0:
        print(i)
    try:
        times = month[val.shop_id][val.item_id].index
        for j in times:
            ans = month[val.shop_id][val.item_id][j]
            #out.iloc[k] = [k, val.shop_id, val.item_id, ans, ans, j]
            ID.append(k)
            shop_id.append(val.shop_id)
            item_id.append(val.item_id)
            count.append(ans)
            dbn.append(j)
            k += 1
            #print([k, val.shop_id, val.item_id, ans, ans, j])
    except:
        lst = []

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [13]:
df = pd.DataFrame({'ID':ID, 'shop_id':shop_id, 'item_id':item_id, 'item_cnt_month':count, 'floor_item_cnt_month':count, 'date_block_num':dbn})
df.head()

,ID,date_block_num,floor_item_cnt_month,item_cnt_month,item_id,shop_id
0,214200,20,1.0,1.0,5037.0,5.0
1,214201,22,1.0,1.0,5037.0,5.0
2,214202,23,2.0,2.0,5037.0,5.0
3,214203,24,2.0,2.0,5037.0,5.0
4,214204,28,1.0,1.0,5037.0,5.0


In [14]:
final = out.append(df, ignore_index=True)
print(final.shape)

(814359, 6)


In [15]:
final.to_csv('complete.csv', index=False)